In [323]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn import *
from scipy import *

from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy.optimize import minimize_scalar

from hyperopt import hp, tpe
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

import re

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

from sklearn.decomposition import TruncatedSVD

from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier

import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.utils.np_utils as kutils

In [303]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [304]:
train = train.sample(frac=0.5)
test = test.sample(frac=0.5)

In [305]:
all_data = pd.concat((train.loc[:,'ps_ind_01':'ps_calc_20_bin'],
                      test.loc[:,'ps_ind_01':'ps_calc_20_bin']))

In [317]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y_train = train['target']

In [307]:
y_train = np_utils.to_categorical(y_train)

In [318]:
X_train.shape

(5952, 57)

In [319]:
def keras_model(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(100, input_dim=57, activation=activation))
    model.add(Dense(100, activation=activation))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [320]:
activation = ["relu"]
optimizer = ["adam"]
out_dim = [300]
nb_epoch = [10]
batch_size = [10]

In [321]:
model = KerasClassifier(build_fn=keras_model, verbose=1)
param_grid = dict(activation=activation, 
                  optimizer=optimizer, 
                  #out_dim=out_dim, 
                  nb_epoch=nb_epoch, 
                  batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [322]:
grid_result = grid.fit(X_train.values, y_train)

Epoch 1/10
3968/3968 [==============================] - 4s - loss: 0.2210 - acc: 0.9650     
Epoch 2/10
3968/3968 [==============================] - 2s - loss: 0.1559 - acc: 0.9693     
Epoch 3/10
3968/3968 [==============================] - 2s - loss: 0.1457 - acc: 0.9708     - ETA: 1s -
Epoch 4/10
3968/3968 [==============================] - 2s - loss: 0.1371 - acc: 0.9708     
Epoch 5/10
3968/3968 [==============================] - 3s - loss: 0.1355 - acc: 0.9708     - ETA: 0s - loss: 0.
Epoch 6/10
3968/3968 [==============================] - 2s - loss: 0.1295 - acc: 0.9708     
Epoch 7/10
3070/3968 [======================>.......] - ETA: 0s - loss: 0.1269 - acc: 0.9704

KeyboardInterrupt: 

In [313]:
print (grid_result.best_score_)
print (grid_result.best_params_)

0.970262091487
{'activation': 'relu', 'batch_size': 10, 'nb_epoch': 10, 'optimizer': 'adam'}


In [314]:
pred = grid_result.predict_proba(X_test.values)

8770/8928 [============================>.] - ETA: 0s

In [316]:
pred[:,1]

array([ 0.08550283,  0.02401491,  0.13563579, ...,  0.04303439,
        0.01667575,  0.00015074], dtype=float32)

In [329]:
def conv_model():
    #nb_epoch = 1 # Change to 100

    batch_size = 128
    img_rows, img_cols = 28, 28

    nb_filters_1 = 32 # 64
    nb_filters_2 = 64 # 128
    nb_filters_3 = 128 # 256
    nb_conv = 3

    cnn = models.Sequential()

    cnn.add(conv.Convolution2D(nb_filters_1, nb_conv, nb_conv,  activation="relu", input_shape=(57,), border_mode='same'))
    cnn.add(conv.Convolution2D(nb_filters_1, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.MaxPooling2D(strides=(2,2)))

    cnn.add(conv.Convolution2D(nb_filters_2, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.Convolution2D(nb_filters_2, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.MaxPooling2D(strides=(2,2)))

    cnn.add(conv.Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same'))
    cnn.add(conv.MaxPooling2D(strides=(2,2)))

    cnn.add(core.Flatten())
    cnn.add(core.Dropout(0.2))
    cnn.add(core.Dense(128, activation="relu")) # 4096
    cnn.add(core.Dense(2, activation="softmax"))
    
    cnn.summary()
    cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    cnn.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1)
    return cnn

In [330]:
model = KerasClassifier(build_fn=conv_model, verbose=1)
param_grid = dict(#activation=activation, 
                  #optimizer=optimizer, 
                  #out_dim=out_dim, 
                  nb_epoch=nb_epoch, 
                  batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [331]:
grid_result = grid.fit(X_train.values, y_train)

/Users/sagashin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(57,), padding="same")`


ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=2